# Setup

In [1]:
from urllib.error import HTTPError

from pyrosm import OSM
import shapely

from utils import *

In [ ]:
states = disp(pd.read_parquet('data/geometry/states.parquet', columns=['code', 'name']))

49 rows x 2 cols; Memory: 0.0 MiB


,code,name
,<object>,<object>
0,AL,Alabama


In [3]:
zones = disp(gpd.read_parquet('data/geometry/zones.parquet'))

143,971 rows x 10 cols; Memory: 12.7 MiB; <Geographic 2D CRS: EPSG:4326>


,geoid,level,county,ua_code,city,aland,awater,popu,labor,geometry
,<object>,<category>,<category>,<category>,<category>,<float32>,<float32>,<int32>,<int32>,<geometry>
0,42007,County,Beaver,69697,Pittsburgh,434.692993,9.329071,164781,84153,"POLYGON ((-80.518963 40.733741, -80.518965 40...."


# Job counts

Download LODES data (https://lehd.ces.census.gov/data) for states which contain at least one CBG of the 50 largest cities.

Download the job count data for 2021 using the Workplace Area Characteristics (WAC) table of the LODES v8 dataset. For Arizona (AR) and Mississippi (MS), the latest data is available for 2018 (for which the dataset version is v7).

To compare results with Access Across America, also download the job count data for 2020.

In [4]:
def download_lodes(year, states=states):
    jobs = []
    for _, r in tqdm(states.iterrows(), total=len(states)):
        cols = D(w_geocode='geoid', C000='All', CE01='Low wage', CR01='White',
            CD01='< high school', CD02='high school')
        url = ('https://lehd.ces.census.gov/data/lodes/' + 
               'LODES8/{0}/wac/{0}_wac_S000_JT00_{1}.csv.gz'
               .format(r.code.lower(), year))
        if r.code == 'MI' and year == 2022:
            url = url.replace('2022', '2021')
        try:
            df = pd.read_csv(url, usecols=list(cols)).rename(columns=cols)
        except HTTPError:
            print(f'URL not available for {r["name"]}')
            continue
        df['Low edu'] = df.pop('< high school') + df.pop('high school')
        df['POC'] = df['All'] - df.pop('White')
        df.geoid = df.geoid.astype(str).str.zfill(15)
        df2 = df.melt('geoid', var_name='kind', value_name='njobs')
        jobs.append(df2.query('njobs > 0').assign(level='Block'))
        for level, nchar in [('Block', 15), ('BG', 12),
                             ('Tract', 11), ('County', 5)]:
            df2 = df.assign(geoid=df.geoid.str[:nchar])
            df2 = df2.groupby('geoid').sum().reset_index()
            df2 = df2.melt('geoid', var_name='kind', value_name='njobs')
            jobs.append(df2.query('njobs > 0').assign(level=level))
    cols = D(geoid=CAT, level=CAT, kind=CAT, njobs=I32)
    jobs = pd.concat(jobs).reset_index(drop=1).astype(cols)[list(cols)]
    jobs.to_parquet(f'data/opport/jobs_{year}.parquet')
    return jobs

# jobs20 = disp(download_lodes(2020)) # 1m25s
# jobs22 = disp(download_lodes(2022)) # 1m25s
jobs22 = disp(pd.read_parquet(
    'data/opport/jobs_2022.parquet',
    filters=[('level', '!=', 'Block')])) # 9s

1,264,563 rows x 4 cols; Memory: 232.3 MiB


,geoid,level,kind,njobs
,<category>,<category>,<category>,<int32>
0,010010201001,BG,All,145


# POIs
Process the Points of Interest (POI) data from OpenStreetMap (OSM) Protocol Buffer Files (PBFs) downloaded on November 23, 2023.

## OSM tags
The [OSM website](https://wiki.openstreetmap.org/wiki/Tags) provides a detailed description of commonly used tags such as `amenity=music_school` or `landuse=farmland`. The traditional method of filtering POIs from this large pool of features using tag information is to specify a list of acceptable tags for POIs, such as the Python library `pyrosm`, which by default uses the tags [amenity, shop, tourism] to filter POIs. However, it should be noted that these OSM tags lack a well-defined typology to identify POIs for which citizens would commonly need access to. By contrast, the [North American Industry Classification System (**NAICS**)](https://www.census.gov/naics) provides an excellent classification system for businesses that can be used to identify POIs of interest. This is also done by POI data vendors such as SafeGraph Inc. or Quadrant.io where POIs include NAICS codes as attributes.

To better align our dataset with such proprietary datasets, we used a mapping between the OSM tags and NAICS codes to only filter relevant POI classes. This mapping table was obtained from [wiki.openstreetmap.org/wiki/NAICS/2022](wiki.openstreetmap.org/wiki/NAICS/2022). Since we are interested only in places where accessibility to retail/direct products/services are available, we only consider a subset of OSM tags in our dataset, including `[amenity, clothes, craft, fast_food, healthcare, second_hand, shop, social_facility]`.

In [4]:
allowed_osm_tags = ['amenity', 'clothes', 'craft', 'fast_food', 'healthcare',
                    'second_hand', 'shop', 'social_facility:for', 'social_facility']

Further, we also define a set of "essential" POIs for special considerations of accessibility measurement:

In [5]:
special_poi_tags = disp(Pdf([(col, tag, value) for col, tag_values in {
    'Groceries': { # grocery and food stores
        'shop': ['convenience', 'department_store', 'greengrocer', 'grocery', 'health_food', 'supermarket']
    },
    'Education': { # educational facilities
        'amenity': ['college', 'dance_school', 'kindergarten', 'language_school', 'music_school', 'preschool', 'school', 'university']
    },
    'Medical': { # medical amenities
        'amenity': ['childcare', 'dentist', 'doctors', 'hospital'],
        'healthcare': ['audiologist', 'blood_bank', 'blood_donation', 'dentist', 'doctor', 'hospital', 'laboratory', 'occupational_therapist', 'physiotherapist', 'podiatrist', 'psychotherapist', 'speech_therapist'],
        'healthcare:speciality': ['chiropractic']
    },
    'Social Support': { # social support facilities
        'social_facility': ['ambulatory_care', 'assisted_living', 'dairy_kitchen', 'food_bank', 'group_home', 'nursing_home', 'outreach', 'shelter', 'soup_kitchen'],
        'social_facility:for': ['drug_addicted']
    }
}.items() for tag, values in tag_values.items() for value in values],
                       columns=['col', 'tag', 'value']))

41 rows x 3 cols; Memory: 0.0 MiB


,col,tag,value
,<object>,<object>,<object>
0,Groceries,shop,convenience


In [6]:
osm_tags = (
    pd.read_csv('data/opport/osm_poi_tags_values.csv')
    .rename(columns=str.lower)
    .dropna(subset='osm')
    .assign(osm_tags=lambda df: df.osm.str.split('OR'))
    .explode('osm_tags')
    .assign(osm_tags=lambda df: df.osm_tags.str.strip().str.split('=')))
osm_tags['tag'], osm_tags['value'] = list(zip(*osm_tags.osm_tags))
osm_tags = (osm_tags[osm_tags.tag.isin(allowed_osm_tags)]
             .merge(special_poi_tags, 'left')
             ['naics description col tag value'.split()])
for col in set(osm_tags.col) - {np.nan}:
    osm_tags[col] = osm_tags.col.eq(col).fillna(False)
osm_tags = disp(osm_tags.drop(columns='col'))

260 rows x 8 cols; Memory: 0.1 MiB


,naics,description,tag,value,Groceries,Medical,Education,Social Support
,<object>,<object>,<object>,<object>,<bool>,<bool>,<bool>,<bool>
0,11291,Apiculture,craft,beekeeper,False,False,False,False


## POI counts
POI counts by category and by zone

In [7]:
def get_pois_by_zone(city, zones=zones, osm_tags=osm_tags):
    city_ = city.lower().replace(' ', '-').replace('.', '')
    osm = OSM(f'data/osm/city/{city_}.osm.pbf')
    df = osm.get_pois(custom_filter={t: True for t in set(osm_tags.tag)})
    # df = pois.copy()
    df.geometry = df.to_crs(CRS_M).centroid
    bgs = filt(zones, city=city, level='BG')[['geoid', 'geometry']]
    df = df.set_index('id').to_crs(bgs.crs).sjoin(bgs)
    tag_cols = [c for c in df.columns if c in set(osm_tags.tag)]
    df = df[['geoid'] + tag_cols].reset_index()
    total_pois = df.groupby('geoid').size().rename('All')
    df = df.melt(['id', 'geoid'], var_name='tag').dropna()
    df = df.merge(osm_tags, on=['tag', 'value'])
    special_cols = ['Groceries', 'Education', 'Medical', 'Social Support']
    df = df.groupby('geoid')[special_cols].sum()
    df = pd.concat([total_pois, df], axis=1).fillna(0).astype(I32)
    df = df.reset_index().melt('geoid', var_name='kind', value_name='npois')
    poi_counts = [df.assign(level='BG')]
    for level, nchar in D(Tract=11, County=5).items():
        poi_counts.append(df.assign(geoid=df.geoid.str[:nchar])
                          .groupby(['geoid', 'kind']).npois.sum()
                          .reset_index().assign(level=level))
    poi_counts = (pd.concat(poi_counts).sort_values(['level', 'geoid'])
                  .reset_index(drop=1)[['geoid', 'level', 'kind', 'npois']]
                  .astype(D(geoid=CAT, level=CAT, kind=CAT, npois=I32)))
    return poi_counts

# pois = get_pois_by_zone('Buffalo'); pois

In [8]:
# osm_pois = []
# for city in (pbar := tqdm(sorted(set(zones.city)))):
#     pbar.set_description(city)
#     osm_pois.append(get_pois_by_zone(city))
# osm_pois = pd.concat(osm_pois).astype(D(geoid=CAT)).reset_index(drop=1)
# disp(osm_pois).to_parquet('data/opport/pois_osm.parquet') # 1h2m
osm_pois = disp(pd.read_parquet('data/opport/pois_osm.parquet'))

639,195 rows x 4 cols; Memory: 17.5 MiB


,geoid,level,kind,npois
,<category>,<category>,<category>,<int32>
0,130131801033,BG,All,11


In [9]:
filt(osm_pois, level='BG').groupby('kind', observed=1).npois.sum().to_frame().T

kind,All,Education,Groceries,Medical,Social Support
npois,1894984,59381,107434,19542,4914


In [11]:
# pd.concat([
#     filt(osm_pois, level='BG').groupby('kind', observed=1).npois.sum().rename('OSM'),
#     sg_pois.merge(filt(zones, level='BG')['geoid']).groupby('kind', observed=1).npois.sum().rename('SafeGraph')
# ], axis=1).assign(Δ=lambda df: df.SafeGraph - df.OSM, pct_Δ=lambda df: df.Δ/df.SafeGraph * 100)

## EV Charging Infrastructure (EVCI)
Data downloaded on Feb 7, 2025 from the [Alternate Fuel Data Center website](https://afdc.energy.gov/fuels/electricity-locations#/analyze?country=US).

In [12]:
cols = {'Latitude': 'lat', 'Longitude': 'lon', 'Open Date': 'open_date',
        'EV Level2 EVSE Num': 'l2', 'EV DC Fast Count': 'fdc'}
evci = (pd.read_csv('data/opport/alt_fuel_stations.csv',
                    usecols=list(cols)).rename(columns=cols)
        .query('(open_date <= "2024-12-31")'))
evci = disp(
    pdf2gdf(evci.fillna(0), 'lon', 'lat', CRS_DEG)
    .sjoin(zones, predicate='within')
    .groupby(['geoid', 'level'], observed=True)
    .agg(evcs=('l2', 'count'), l2=('l2', 'sum'), fdc=('fdc', 'sum'))
    .astype(I32).reset_index()
    .rename(columns=D(evcs='Stations', l2='Level 2', fdc='Fast DC'))
    .melt(['geoid', 'level'], var_name='kind', value_name='opport')
)

84,978 rows x 4 cols; Memory: 9.9 MiB


,geoid,level,kind,opport
,<object>,<category>,<object>,<int32>
0,04013,County,Stations,958


## Combine job & POI counts

In [13]:
opport = pd.concat([
    jobs22.assign(kind='Jobs: ' + jobs22.kind.astype(str))
    .rename(columns=D(njobs='opport')),
    osm_pois.assign(kind='POIs: ' + osm_pois.kind.astype(str))
    .rename(columns=D(npois='opport')),
    evci.assign(kind='EVCI: ' + evci.kind)
]).reset_index(drop=True).astype(D(geoid=CAT, kind=CAT, opport=I32))
disp(opport).to_parquet('data/opport/opportunities.parquet')
# opport = disp(pd.read_parquet('data/opport/opportunities.parquet'))

1,988,736 rows x 4 cols; Memory: 142.0 MiB


,geoid,level,kind,opport
,<category>,<object>,<category>,<int32>
0,010010201001,BG,Jobs: All,145


In [14]:
opport[opport.kind.str[:4] != 'EVCI'].groupby('kind', observed=1).opport.sum().sort_index()

kind
Jobs: All               435419592
Jobs: Low edu           136558113
Jobs: Low wage           77543463
Jobs: POC               100459104
POIs: All                 5684952
POIs: Education            178143
POIs: Groceries            322302
POIs: Medical               58626
POIs: Social Support        14742
Name: opport, dtype: int32

## City summary statistics

In [16]:
urba = disp(pd.read_parquet('data/geometry/urban_zones.parquet'))

275,549 rows x 4 cols; Memory: 17.7 MiB


,code,name,level,geoid
,<category>,<category>,<category>,<object>
0,58600,"Montgomery, AL",County,01001


In [17]:
bgs2 = disp(
    filt(zones.drop(columns='geometry'), level='BG')
    .merge(urba[['name', 'geoid']], on='geoid')
    .merge(filt(jobs22, kind='All', level='BG'), 'left')
    .merge(filt(osm_pois, kind='All', level='BG'), 'left')
    .merge(filt(evci, kind='Stations', level='BG')
           .rename(columns=D(opport='nevcs')), 'left')
    .astype(D(county=str, ua_code=str, city=str, name=str))
    .fillna(0))

106,044 rows x 12 cols; Memory: 34.5 MiB


,geoid,county,ua_code,city,aland,awater,popu,labor,name,njobs,npois,nevcs
,<object>,<object>,<object>,<object>,<float32>,<float32>,<int32>,<int32>,<object>,<float64>,<float64>,<float64>
0,280330705233,DeSoto,56116,Memphis,1.008998,0.0,3924,1832,"Memphis, TN--MS--AR",0.0,13.0,0.0


In [18]:
df = bgs2.groupby(['ua_code', 'name', 'city']).agg(D(
    geoid='count', county=set, aland='sum', awater='sum',
    popu='sum', njobs='sum', npois='sum', nevcs='sum'
)).reset_index().rename(columns=D(geoid='nBG'))
ntract = (bgs2.assign(geoid=bgs2.geoid.str[:11])
          .groupby(['city', 'geoid']).size().reset_index()
          .groupby('city').geoid.count().rename('nTract').reset_index())
df = df.merge(ntract, on='city')
df['popdens'] = df.popu / df.aland
df['nCounty'] = df.county.apply(len)
df.county = [', '.join(x) for x in df.county]
df.index = df.popu.rank(ascending=0).astype(I16).rename('rank')
for col in ['popu', 'njobs', 'npois']:
    df[col] /= 1000
df = df[df.nBG > 1].sort_index().reset_index().astype(D(nevcs=I16))
summary_stats = df['rank city nBG nTract nCounty popu aland popdens njobs npois nevcs'.split()]
summary_stats

,rank,city,nBG,nTract,nCounty,popu,aland,popdens,njobs,npois,nevcs
0,1,New York,14305,4790,24,18718.361,4442.073242,4213.879416,9129.811,237.604,2604
1,2,Los Angeles,8085,2912,4,12329.329,2039.094849,6046.471555,6127.292,80.690,5853
2,3,Chicago,6276,2143,10,8609.691,3252.777100,2646.873959,4285.828,110.545,1294
3,4,Miami,3906,1478,4,6084.318,2790.271484,2180.546959,2701.292,34.908,1380
4,5,Houston,3447,1354,7,5847.375,2657.120605,2200.643429,2810.097,47.555,738
5,6,Dallas,3446,1350,10,5742.504,2489.686279,2306.517109,3402.482,48.922,963
6,7,Philadelphia,4328,1471,12,5724.087,2798.585938,2045.349733,2817.087,51.874,914
7,8,Washington,3319,1231,13,5149.414,1742.771729,2954.726609,2730.361,76.841,1751
8,9,Atlanta,3353,1299,18,5021.762,3172.847412,1582.730383,2657.837,44.882,1482
9,10,Boston,3281,1041,8,4454.117,2327.142090,1913.985837,2581.741,59.416,2506


In [19]:
df2 = df[['ua_code', 'name', 'city', 'county']].copy()
df2.insert(0, 'rank', df2.index + 1)
df2 = df2.rename(columns=D(ua_code='area_code', name='area_name',
                           city='chief_city', county='constituent_counties'))
disp(df2).to_csv('data/geometry/city_ua_counties.csv', index=False)

50 rows x 5 cols; Memory: 0.0 MiB


,rank,area_code,area_name,chief_city,constituent_counties
,<int64>,<object>,<object>,<object>,<object>
0,1,63217,"New York--Jersey City--Newark, NY--NJ",New York,"Monmouth, Hudson, Nassau, Sussex, Union, Merce..."
